In [1]:
!wget https://hydranets-data.s3.eu-west-3.amazonaws.com/hydranets-data.zip && unzip -q hydranets-data.zip && mv hydranets-data/* . && rm hydranets-data.zip && rm -rf hydranets-data

--2024-10-10 10:34:22--  https://hydranets-data.s3.eu-west-3.amazonaws.com/hydranets-data.zip
Resolving hydranets-data.s3.eu-west-3.amazonaws.com (hydranets-data.s3.eu-west-3.amazonaws.com)... 3.5.225.114, 52.95.155.60
Connecting to hydranets-data.s3.eu-west-3.amazonaws.com (hydranets-data.s3.eu-west-3.amazonaws.com)|3.5.225.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 110752264 (106M) [application/zip]
Saving to: ‘hydranets-data.zip’

hydranets-data.zip  100%[===================>] 105.62M  23.6MB/s    in 5.2s    

2024-10-10 10:34:27 (20.1 MB/s) - ‘hydranets-data.zip’ saved [110752264/110752264]



In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
from torch.autograd import Variable

In [3]:
class HydraNet(nn.Module):
    def __init__(self):
        #super(HydraNet, self).__init__() # Python2
        super().__init__() # Python 3
        self.num_tasks = 2
        self.num_classes = 6

In [4]:
hydranet = HydraNet()

In [5]:
def conv3x3(in_channels, out_channels, stride=1, dilation=1, groups=1, bias=False):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=dilation, bias=bias, grpops=groups)

In [6]:
def conv1x1(in_channels, out_channels, stride=1, dilation=0, groups=1, bias=False):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=dilation, bias=bias, grpops=groups)

In [7]:
def batchnorm(num_features):
    return nn.BatchNorm2d(num_features, affine=True, eps=1e-5, momentum=0.1)

In [12]:
def convbnrelu(in_channels, out_channels, kernel_size, stride=1, groups=1, act=True):
  if act:
    return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=int(kernel_size/2.0), groups=groups, bias=False),
                         batchnorm(out_channels),
                         nn.ReLU6(inplace=True))
  else:
    return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=int(kernel_size/2.0), groups=groups, bias=False),
                         batchnorm(out_channels))

In [13]:
class InvertedResidualBlock(nn.Module):
  def __init__(self, in_planes, out_planes, expansion_factor, stride=1):
    super().__init__()
    intermed_planes = in_planes * expansion_factor
    self.residual = (in_planes == out_planes) and (stride == 1)
    self.output = nn.Sequential(convbnrelu(in_planes, intermed_planes, 1),
                                convbnrelu(intermed_planes, intermed_planes, 3, stride=stride, groups=intermed_planes),
                                convbnrelu(intermed_planes, out_planes, 1, act=False))
  def forward(self, x):
    if self.residual:
      return x + self.output(x)
    else:
      return self.output(x)

In [14]:
def define_mobilenet(self):
  mobilenet_config = [[1, 16, 1, 1],
                      [6, 24, 2, 2],
                      [6, 32, 3, 2],
                      [6, 64, 4, 2],
                      [6, 96, 3, 1],
                      [6, 160, 3, 2],
                      [6, 320, 1, 1]]

  self.in_channels = 32
  self.num_layers = len(mobilenet_config)
  self.layers1 = convbnrelu(3, self.in_channels, kernel_size=3, stride=2)
  c_layer = 2

  for t, c, n, s in mobilenet_config:
    layers = []
    for idx in range(n):
      layers.append(InvertedResidualBlock(self.in_channels, c, expansion_factor=t, stride=s if idx == 0 else 1))
      self.in_channels = c
    setattr(self, "layer{}".format(c_layer), nn.Sequential(*layers))
    c_layer += 1

HydraNet.define_mobilenet = define_mobilenet

In [15]:
hydranet.define_mobilenet()

In [16]:
print(hydranet)

HydraNet(
  (layers1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (layer2): Sequential(
    (0): InvertedResidualBlock(
      (output): Sequential(
        (0): Sequential(
          (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Sequential(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (2): Sequential(
          (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affi